In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import mean_absolute_error
import pydmd
import backtrader as bt
import yfinance as yf

import backtrader.analyzers as btanalyzers
from datetime import datetime
import backtrader.feeds as btfeeds


In [6]:
sym = pd.read_csv("DATA/nse100_SYM.csv",header=None) 
sym = sym[0].tolist()
SYM =[ i + ".NS" for i in sym]

In [22]:
df = pd.DataFrame()
matx = pd.read_csv("DATA/NSE100.csv",header=None) #VARIABLE 1
matx = pd.DataFrame(matx.transpose())
matx = matx.fillna(0)

In [8]:
cerebro = bt.Cerebro()
 
stocks = SYM
for s in stocks: 
    data = yf.download(s, start="2012-01-01", end="2023-01-17") 
    data.to_csv('data.csv')
    data = btfeeds.YahooFinanceCSVData(dataname='data.csv')
    cerebro.adddata(data, name = s)

           

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [27]:
def dmd_pred(j,i=16) :
    # creating the matrix mat 
    # i days are trained j th day from last 
    mat = matx.iloc[:, -(i)+j:j]
    mat=mat.to_numpy()


    # dmd 
    dmd = pydmd.DMD(svd_rank=mat.shape[1])
    dmd.fit(mat)

    d_ind = np.argmax(np.abs(dmd.amplitudes.real))
    print(np.amax(dmd.modes[:,d_ind].real))

    #if any(np.abs(dmd.eigs.real)>1) and dmd.amplitudes.real[d_ind] > 0 :
    if np.abs(dmd.eigs.real[d_ind])>1 and dmd.eigs.imag[d_ind] == 0 and dmd.amplitudes.real[d_ind] > 0 :
        print("^^^^ GrowinG ^^^^")
        d_ind = np.argmax(np.abs(dmd.amplitudes.real))
        return(d_ind)

        
 
    else:
        return(0)



In [23]:
SYM

['KARURVYSYA.NS',
 'RELIANCE.NS',
 'TCS.NS',
 'HDFCBANK.NS',
 'INFY.NS',
 'HINDUNILVR.NS',
 'ICICIBANK.NS',
 'SBIN.NS',
 'HDFC.NS',
 'DICIND.NS',
 'BHARTIARTL.NS',
 'ADANIENT.NS',
 'ATGL.NS',
 'ITC.NS',
 'BAJFINANCE.NS',
 'KOTAKBANK.NS',
 'ADANIGREEN.NS',
 'LT.NS',
 'ADANITRANS.NS',
 'HCLTECH.NS',
 'AXISBANK.NS',
 'ASIANPAINT.NS',
 'MARUTI.NS',
 'DMART.NS',
 'SUNPHARMA.NS',
 'BAJAJFINSV.NS',
 'WIPRO.NS',
 'TITAN.NS',
 'ULTRACEMCO.NS',
 'NESTLEIND.NS',
 'JSWSTEEL.NS',
 'ONGC.NS',
 'ADANIPORTS.NS',
 'M&M.NS',
 'NTPC.NS',
 'POWERGRID.NS',
 'HINDZINC.NS',
 'TATASTEEL.NS',
 'TATAMOTORS.NS',
 'COALINDIA.NS',
 'SBILIFE.NS',
 'HDFCLIFE.NS',
 'PIDILITIND.NS',
 'LTIM.NS',
 'VEDL.NS',
 'IOC.NS',
 'HINDALCO.NS',
 'GRASIM.NS',
 'ADANIPOWER.NS',
 'SIEMENS.NS',
 'BRITANNIA.NS',
 'AMBUJACEM.NS',
 'BAJAJ-AUTO.NS',
 'TECHM.NS',
 'DABUR.NS',
 'INDUSINDBK.NS',
 'BANKBARODA.NS',
 'GODREJIND.NS',
 'DLF.NS',
 'DIVISLAB.NS',
 'SHREECEM.NS',
 'EICHERMOT.NS',
 'CIPLA.NS',
 'HAL.NS',
 'INDIGO.NS',
 'BPCL.NS',
 '

In [30]:
class MyStrategy(bt.Strategy):
 
     
    def __init__(self):
        self.ind = bt.indicators.SimpleMovingAverage(self.data.close, period=10)
 
    def next(self):
        for i, d in enumerate(self.datas):
            stocks_to_buy = dmd_pred(j) # call the function to get the stocks to buy
            for stock in stocks_to_buy:
                self.buy(data = SYM[stock]) # buy 100 shares of each stock


In [7]:
class MaCrossStrategy(bt.Strategy):
 
     
    def __init__(self):
        self.ind = bt.indicators.SimpleMovingAverage(self.data.close, period=10)
 
    def next(self):
        for i, d in enumerate(self.datas):
            if not self.getposition(d).size:
                if self.crossovers[i] > 0: 
                    self.buy(data = d)
            elif self.crossovers[i] < 0: 
                self.close(data = d)
 


In [31]:
cerebro.addstrategy(MaCrossStrategy)
 
cerebro.broker.setcash(1000000.0)
 
cerebro.addsizer(bt.sizers.PercentSizer, percents = 1.5)
 
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = "sharpe")
cerebro.addanalyzer(btanalyzers.Returns,     _name = "returns")
cerebro.addanalyzer(btanalyzers.Transactions, _name = "trans")
 
back = cerebro.run()
 
cerebro.broker.getvalue()


TypeError: dmd_pred() missing 1 required positional argument: 'j'

In [21]:
back[0].analyzers.returns.get_analysis()['rnorm100']


-inf

In [ ]:
back[0].analyzers.returns.get_analysis()['rnorm100']
back[0].analyzers.sharpe.get_analysis()
back[0].analyzers.trans.get_analysis()

In [7]:
cerebro.broker.getvalue()


2309349.409298027